In [1]:
import logging
logging.captureWarnings(False)
logging.getLogger('py.warnings').setLevel(logging.ERROR)
from dask.distributed import Client, progress
client = Client(n_workers=5, threads_per_worker=4, memory_limit='100GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 20,Total memory: 465.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58704,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 465.66 GiB
Comm: tcp://127.0.0.1:58722,Total threads: 4
Dashboard: http://127.0.0.1:58723/status,Memory: 93.13 GiB
Nanny: tcp://127.0.0.1:58708,


In [2]:
import numpy as np
import torch
import subprocess
import sys
import torch_harmonics as th
import torch_harmonics.distributed as dist
import pandas as pd
import matplotlib.pyplot as plt
from subs1_utils import *

In [3]:
# For MacOS is higher than 12.3+
if torch.backends.mps.is_available():
    print("Running on GPU")
#    print("Running on CPU")
    device = torch.device("mps")
#    device = torch.device("cpu")
    print("MPS is activated:",torch.backends.mps.is_built()) # Was the current version of PyTorch built with MPS activated?
else:
    print("Running  on CPU")
    device = torch.device("cpu")

device

Running on GPU
MPS is activated: True


device(type='mps')

In [4]:
### This cell initializes the model
###
###     First Define all spectral grids
###
zw = 63 # zonal wave number
mw = 63 # meridional wave number
kmax = 26
imax = 192
jmax = 96
steps_per_day = 216*1.5 ### Changing this number impliws time step changes and should
#                       be implemented carefully
rescale = 15 # rescale interval for bred-vector approach to linear model
###
#
#
# provide experiment name and data path for writing out data
# datapath may need to be edited for your system
#
expname = 'TestLinearT63L26F_Dist'
foo = str(subprocess.check_output(['whoami']))
end = len(foo) - 3
uname = foo[2:end]
datapath = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/'+expname+'/'
toffset = 0
#
# If restart see below settings
#
#toffset = 1800*1 # toffset is the number of days that have already run
#datapath_init = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/Restarts/Total/' # Location in restarts
datapath_init = datapath #set equal to datapath if restarting in same directory
#
if ( toffset == 0): # Cold Start
    subprocess.call(['rm','-r', datapath])
    subprocess.check_output(['mkdir', datapath])

In [5]:
# Get the Gaussian latitudes and equally spaced longitudes
#
cost_lg, wlg, lats = precompute_latitudes(jmax)
lats = 90-180*lats/(np.pi)
lons = np.linspace(0.0,360.0-360.0/imax,imax)
#
# Instantiate grid to spectral (dsht) and spectral to grid (disht) distibuted transforms
#
vsht = th.RealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
dsht = dist.DistributedRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
disht = dist.DistributedInverseRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
dvsht = dist.DistributedRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
divsht = dist.DistributedInverseRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)

In [6]:
#
#
#
# Initialize spectral fields (at rest or to be read in)
def initialize(temp_newton,lnpsclim,kmax,mw,zw):
    for k in range (kmax):
        tmn1[k] = tmn1[k] + temp_newton[k]
        tmn2[k] = tmn2[k] + temp_newton[k]
        tmn3[k] = tmn3[k] + temp_newton[k]
    qmn1 = lnpsclim
    qmn2 = lnpsclim
    qmn3 = lnpsclim
    return tmn1,tmn2,tmn3,qmn1,qmn2,qmn3

In [7]:
#
# Initialization: Could read spectral restarts, or could
#              start at rest. If wanting to use grid point see
#              jupyter notebook preprocess
#              
#
# Implement at rest initial condition, but need coriolis since
# model predicts total vorticity
#
coriolis = np.broadcast_to([(4.0*np.pi/86400)*np.sin(lats[j,np.newaxis]*np.pi/180.0) for j in range(jmax)], (jmax, imax))
#
#
# Initialize spectral fields (at rest or to be read in)
# Need to read in background temperature data for Newtonian
# Relaxation and possible initialization, see preprocess
# for how to change source data or formulation.
#
temp_newton = torch.load('temp.spectral.pt') # newtonian 
#                                    relaxation temperature, spectral
lnpsclim = torch.load('lnps.spectral.pt') # lnps climatology for damping
#
# Read Climatology on Gausian Grid
#
tclim_gg = torch.load('tsig.ggrid.pt')
vortclim_gg = torch.load('vortsig.ggrid.pt')
divclim_gg = torch.load('divsig.ggrid.pt')
uclim_gg = torch.load('usig.ggrid.pt')
vclim_gg = torch.load('vsig.ggrid.pt')
dxqclim_gg = torch.load('dxq_gg.ggrid.pt')
dyqclim_gg = torch.load('dyq_gg.ggrid.pt')
#
divclim = dsht(divclim_gg)
vortclim = dsht(vortclim_gg)
tclim = dsht(tclim_gg)
#
zmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
zmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    zmn1 = torch.load(datapath_init+'zmn1.spectral.pt')
    zmn2 = torch.load(datapath_init+'zmn2.spectral.pt')
zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
dmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128) 
dmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    dmn1 = torch.load(datapath_init+'dmn1.spectral.pt')
    dmn2 = torch.load(datapath_init+'dmn2.spectral.pt')
dmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
tmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    tmn1 = torch.load(datapath_init+'tmn1.spectral.pt') 
    tmn2 = torch.load(datapath_init+'tmn2.spectral.pt')
tmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
wmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
qmn1 = torch.zeros((mw,zw),dtype=torch.complex128)
qmn2 = torch.zeros((mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    qmn1 = torch.load(datapath_init+'qmn1.spectral.pt')
    qmn2 = torch.load(datapath_init+'qmn2.spectral.pt')
qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
#
if ( toffset == 0 ): # Only do this if cold start
    tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 =\
    initialize(tclim,lnpsclim,kmax,mw,zw)
#
# Topography data - this should be spectral data or can be
#                        initialized to zero. If grid point data
#                        is desired see preprocess for how to
#                        convert to spectral. 
#
# Setting topography to zero here
#
phismn = torch.zeros((mw,zw),dtype=torch.complex128)
#
#
#
# Adding heating here see preprocess.ipynb
#
heat = torch.load('heat_linear.ggrid.pt')
#
#
#

In [8]:
###
### Constants, parameters, vertical differencing parameters,
### matricies for geopotential height, etc ...
###
delsig, si, sl, sikap, slkap, cth1, cth2, r1b, r2b = bscst(kmax)
### The above code is in subs1_utils.py - vertical structure related
### This code would need to be changed if the vertical resolution
### is changed - could be done by simply specifying delsig in bscst
###
amtrx, cmtrx, dmtrx = mcoeff(kmax,si,sl,slkap,r1b,r2b,delsig)
### The above code is for geopotential height and implicit scheme
### in subs1_utils.py but unlikely any changes would be needed
emtrx = inv_em(dmtrx,steps_per_day,kmax,mw,zw)
### The above code
### emtrix is used in the implicit time scheme, computed once here to save cpu time
### changes unlikely

In [ ]:
#### Preprocessing is complete - now time to run model
#
#
# The Model Runs in 30-day chuncks - need to specify how many 30-day chunks to run
tl = 30 ##### tl is the chunk size - typically 30 days, but for testing 3 is reasonable
#
# Suggested ichunk for time dependent models: 120
#
ae = 6.371E+06 # Earth radius
tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
zmnt = tmnt.detach().clone()
dmnt = tmnt.detach().clone()
qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
wmnt = tmnt.detach().clone()
#
ddtdiv = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
ddtvort = ddtdiv.detach().clone()
ttend = ddtdiv.detach().clone()
#
vort = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
div = ddtdiv .detach().clone()
temp = ddtdiv.detach().clone()
qdot = ddtdiv.detach().clone()
#
times = pd.date_range(start = '1950-01-01', end='2100-01-01', freq='D')
# 
ichunk = 60
#
idays = tl * ichunk
#
#
#
#
# Begin Time Loop
#
ii = 0
savedat = 0
daycount = 0
total_days = 0
nstep = idays*steps_per_day
irescale = 0
while ii < nstep:
    ii = ii + 1
    savedat = savedat + 1
    zmnt[daycount] = zmnt[daycount] + zmn1/steps_per_day
    dmnt[daycount] = dmnt[daycount] + dmn1/steps_per_day
    tmnt[daycount] = tmnt[daycount] + tmn1/steps_per_day
    qmnt[daycount] = qmnt[daycount] + qmn1/steps_per_day
    wmnt[daycount] = wmnt[daycount] + wmn1/steps_per_day
    if (savedat == steps_per_day): # post processing
        #
        # Call Postprocessing Routine as needed
        #
        print((dmn1[10,2,2],dmn1[9,2,1]))
        daycount = daycount + 1
        total_days = total_days + 1
        print(['Day = ',total_days])
        if (daycount == tl):
            times_30day = times[total_days-tl+toffset:total_days+toffset]
            postprocessing(disht,divsht,zmnt,dmnt,tmnt,qmnt,wmnt,\
                           phismn,amtrx,times_30day,mw,zw,\
                           kmax,imax,jmax,sl,lats,lons,tl,datapath)
            tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            zmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            dmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
            wmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            daycount = 0
        savedat = 0
        irescale = irescale + 1
        scale = 1.0e+03
        if ( irescale == rescale ): ### Applying bred vector approach here
            irescale = 0
            zmn1 = zmn1/scale
            zmn2 = zmn2/scale
            dmn1 = dmn1/scale
            dmn2 = dmn2/scale
            tmn1 = tmn1/scale
            tmn2 = tmn2/scale
            qmn1 = qmn1/scale
            qmn2 = qmn2/scale
        #
    #
    # Run model for one time step
    #
    # Spectral to grid transformation of needed fields:
    #   Vorticity, divergence, temperature, U, V, 
    #   grad(ln(Ps)), Q prescibed heating
    #
    #
    vort = disht(zmn2) ### This is the relative vorticity
    div = disht(dmn2)
    temp = disht(tmn2)
    qdot = disht(wmn2)
    u,v = uv(divsht,zmn2,dmn2,mw,zw,kmax,imax,jmax)
    dxq,dyq = gradq(divsht,qmn2,mw,zw,imax,jmax)
    #
    # Stack grid variables climo first[0] perturbation second[1]
    #
    vort2 = torch.stack((vortclim_gg,vort)) # vortclim is the absolute vorticity
    div2 = torch.stack((divclim_gg,div))
    temp2 = torch.stack((tclim_gg,temp))
    u2 = torch.stack((uclim_gg,u))
    v2 = torch.stack((vclim_gg,v))
    dxq2 = torch.stack((dxqclim_gg,dxq))
    dyq2 = torch.stack((dyqclim_gg,dyq))
    #
    # Non-Linear products
    #
    a,b,e,ut,vt,ri,wj,cbar,dbar = nlprod_prescribed_mean_linear(u2,v2,vort2,div2,temp2,dxq2,dyq2,heat,coriolis,delsig,si,sikap,slkap,\
                                                                r1b,r2b,cth1,cth2,cost_lg,kmax,imax,jmax)
    #
    #
    # Grid to spectral transformation of nlprod results
    #
    ddtdiv,ddtvort = vortdivspec(vsht,a,b,kmax,mw,zw)
    zmn3 = - ddtvort
    dmn3 = ddtdiv - lap_sht(dsht,e,mw,zw)
    _,ttend = vortdivspec(vsht,ut,vt,kmax,mw,zw)
    #
    tmn3 = -ttend + dsht(ri)
    wmn3 = dsht(wj) ### Prescribed heating converted to spectral
    qmn3 = -dsht(cbar[1]) ### Only cbar here since dbar is included in implicit or explicit
    # 
    # Diffusion, Damping, Implicit or Explicit time differencing, Time filter
    #
    zmn3,dmn3,tmn3 = diffsn(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,mw,zw)
    #
    zmn3,dmn3,tmn3,qmn3 = damp_prescribed_mean(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,qmn1,qmn3,kmax,mw,zw)
    #
    dt = 86400.0/steps_per_day
    #
    zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 = \
                        explicit(dt,amtrx,cmtrx,dmtrx,emtrx,\
                        zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,\
                        tmn3,wmn1,wmn2,wmn3,qmn1,qmn2,qmn3,phismn,\
                        delsig,kmax,mw,zw)
    ####
    ## Reset zmn3, dmn3, tmn3,wmn3 & qmn3
    ###
    zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    dmn3 = zmn3.detach().clone()
    tmn3 = zmn3.detach().clone()
    wmn3 = zmn3.detach().clone()
    qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
# Done

(tensor(3.0856e-06-1.4216e-06j, dtype=torch.complex128), tensor(-5.6758e-07-6.3930e-07j, dtype=torch.complex128))
['Day = ', 1]
(tensor(-1.1159e-06+9.7545e-07j, dtype=torch.complex128), tensor(7.7019e-07-2.5066e-06j, dtype=torch.complex128))
['Day = ', 2]
(tensor(-8.1799e-09-7.2468e-07j, dtype=torch.complex128), tensor(7.5682e-07-7.0144e-07j, dtype=torch.complex128))
['Day = ', 3]
(tensor(-2.3752e-06+3.7554e-08j, dtype=torch.complex128), tensor(-3.7150e-07+1.6548e-06j, dtype=torch.complex128))
['Day = ', 4]
(tensor(-2.3869e-07-6.7382e-07j, dtype=torch.complex128), tensor(-2.1010e-06-2.4023e-07j, dtype=torch.complex128))
['Day = ', 5]
(tensor(-2.0744e-06-2.3738e-06j, dtype=torch.complex128), tensor(-1.0156e-06-2.1584e-06j, dtype=torch.complex128))
['Day = ', 6]
(tensor(3.7052e-07-2.2541e-06j, dtype=torch.complex128), tensor(4.2784e-07-2.7009e-07j, dtype=torch.complex128))
['Day = ', 7]
(tensor(-5.4750e-07-1.3713e-06j, dtype=torch.complex128), tensor(6.3265e-08+2.9185e-07j, dtype=torch.c

(tensor(-4.2098e-07-9.6436e-07j, dtype=torch.complex128), tensor(-3.5066e-08-1.4262e-07j, dtype=torch.complex128))
['Day = ', 65]
(tensor(-5.8641e-07-1.1363e-06j, dtype=torch.complex128), tensor(-2.3912e-08-1.5225e-07j, dtype=torch.complex128))
['Day = ', 66]
(tensor(-7.3707e-07-1.1398e-06j, dtype=torch.complex128), tensor(-3.1144e-08-1.8762e-07j, dtype=torch.complex128))
['Day = ', 67]
(tensor(-7.1525e-07-9.7656e-07j, dtype=torch.complex128), tensor(-4.0366e-08-1.8753e-07j, dtype=torch.complex128))
['Day = ', 68]
(tensor(-5.0191e-07-9.8750e-07j, dtype=torch.complex128), tensor(-2.4662e-08-1.9389e-07j, dtype=torch.complex128))
['Day = ', 69]
(tensor(-4.4279e-07-1.1795e-06j, dtype=torch.complex128), tensor(-6.5703e-08-2.1330e-07j, dtype=torch.complex128))
['Day = ', 70]
(tensor(-4.8951e-07-1.2335e-06j, dtype=torch.complex128), tensor(-1.1172e-07-2.1176e-07j, dtype=torch.complex128))
['Day = ', 71]
(tensor(-3.7783e-07-1.1436e-06j, dtype=torch.complex128), tensor(-1.1282e-07-2.3037e-07j, 

(tensor(-7.1540e-07-9.7642e-07j, dtype=torch.complex128), tensor(-4.0250e-08-1.8755e-07j, dtype=torch.complex128))
['Day = ', 128]
(tensor(-5.0203e-07-9.8776e-07j, dtype=torch.complex128), tensor(-2.4529e-08-1.9381e-07j, dtype=torch.complex128))
['Day = ', 129]
(tensor(-4.4271e-07-1.1803e-06j, dtype=torch.complex128), tensor(-6.5695e-08-2.1307e-07j, dtype=torch.complex128))
['Day = ', 130]
(tensor(-4.8909e-07-1.2346e-06j, dtype=torch.complex128), tensor(-1.1198e-07-2.1149e-07j, dtype=torch.complex128))
['Day = ', 131]
(tensor(-3.7723e-07-1.1440e-06j, dtype=torch.complex128), tensor(-1.1333e-07-2.3031e-07j, dtype=torch.complex128))
['Day = ', 132]
(tensor(-2.4539e-07-1.2449e-06j, dtype=torch.complex128), tensor(-1.5523e-07-2.1952e-07j, dtype=torch.complex128))
['Day = ', 133]
(tensor(-2.1767e-07-1.3529e-06j, dtype=torch.complex128), tensor(-1.5982e-07-2.3267e-07j, dtype=torch.complex128))
['Day = ', 134]
(tensor(-2.7088e-07-1.4845e-06j, dtype=torch.complex128), tensor(-1.5084e-07-2.2639

(tensor(-4.8908e-07-1.2346e-06j, dtype=torch.complex128), tensor(-1.1199e-07-2.1148e-07j, dtype=torch.complex128))
['Day = ', 191]
(tensor(-3.7722e-07-1.1440e-06j, dtype=torch.complex128), tensor(-1.1334e-07-2.3031e-07j, dtype=torch.complex128))
['Day = ', 192]
(tensor(-2.4539e-07-1.2448e-06j, dtype=torch.complex128), tensor(-1.5523e-07-2.1954e-07j, dtype=torch.complex128))
['Day = ', 193]
(tensor(-2.1770e-07-1.3528e-06j, dtype=torch.complex128), tensor(-1.5981e-07-2.3269e-07j, dtype=torch.complex128))
['Day = ', 194]
(tensor(-2.7095e-07-1.4845e-06j, dtype=torch.complex128), tensor(-1.5080e-07-2.2640e-07j, dtype=torch.complex128))
['Day = ', 195]
(tensor(2.9567e-07-6.3008e-07j, dtype=torch.complex128), tensor(6.6558e-08-6.7375e-08j, dtype=torch.complex128))
['Day = ', 196]
(tensor(-9.5554e-08-9.3223e-07j, dtype=torch.complex128), tensor(1.8942e-08-1.6881e-08j, dtype=torch.complex128))
['Day = ', 197]
(tensor(-4.5622e-07-8.8174e-07j, dtype=torch.complex128), tensor(-3.1568e-08-8.6623e-0

(tensor(-2.1770e-07-1.3528e-06j, dtype=torch.complex128), tensor(-1.5981e-07-2.3269e-07j, dtype=torch.complex128))
['Day = ', 254]
(tensor(-2.7095e-07-1.4845e-06j, dtype=torch.complex128), tensor(-1.5080e-07-2.2640e-07j, dtype=torch.complex128))
['Day = ', 255]
(tensor(2.9567e-07-6.3008e-07j, dtype=torch.complex128), tensor(6.6558e-08-6.7375e-08j, dtype=torch.complex128))
['Day = ', 256]
(tensor(-9.5554e-08-9.3223e-07j, dtype=torch.complex128), tensor(1.8942e-08-1.6881e-08j, dtype=torch.complex128))
['Day = ', 257]
(tensor(-4.5622e-07-8.8174e-07j, dtype=torch.complex128), tensor(-3.1568e-08-8.6623e-08j, dtype=torch.complex128))
['Day = ', 258]
(tensor(-4.5959e-07-8.1709e-07j, dtype=torch.complex128), tensor(-3.8242e-08-1.5949e-07j, dtype=torch.complex128))
['Day = ', 259]
(tensor(-4.2093e-07-9.6432e-07j, dtype=torch.complex128), tensor(-3.5105e-08-1.4263e-07j, dtype=torch.complex128))
['Day = ', 260]
(tensor(-5.8640e-07-1.1361e-06j, dtype=torch.complex128), tensor(-2.3931e-08-1.5230e-0

...

(tensor(-9.5554e-08-9.3223e-07j, dtype=torch.complex128), tensor(1.8942e-08-1.6881e-08j, dtype=torch.complex128))
['Day = ', 1262]
(tensor(-4.5622e-07-8.8174e-07j, dtype=torch.complex128), tensor(-3.1568e-08-8.6623e-08j, dtype=torch.complex128))
['Day = ', 1263]
(tensor(-4.5959e-07-8.1709e-07j, dtype=torch.complex128), tensor(-3.8242e-08-1.5949e-07j, dtype=torch.complex128))
['Day = ', 1264]
(tensor(-4.2093e-07-9.6432e-07j, dtype=torch.complex128), tensor(-3.5105e-08-1.4263e-07j, dtype=torch.complex128))
['Day = ', 1265]
(tensor(-5.8640e-07-1.1361e-06j, dtype=torch.complex128), tensor(-2.3931e-08-1.5230e-07j, dtype=torch.complex128))
['Day = ', 1266]
(tensor(-7.3715e-07-1.1396e-06j, dtype=torch.complex128), tensor(-3.1101e-08-1.8768e-07j, dtype=torch.complex128))
['Day = ', 1267]
(tensor(-7.1540e-07-9.7642e-07j, dtype=torch.complex128), tensor(-4.0247e-08-1.8755e-07j, dtype=torch.complex128))
['Day = ', 1268]
(tensor(-5.0203e-07-9.8777e-07j, dtype=torch.complex128), tensor(-2.4527e-08-

(tensor(-4.2093e-07-9.6432e-07j, dtype=torch.complex128), tensor(-3.5105e-08-1.4263e-07j, dtype=torch.complex128))
['Day = ', 1325]
(tensor(-5.8640e-07-1.1361e-06j, dtype=torch.complex128), tensor(-2.3931e-08-1.5230e-07j, dtype=torch.complex128))
['Day = ', 1326]
(tensor(-7.3715e-07-1.1396e-06j, dtype=torch.complex128), tensor(-3.1101e-08-1.8768e-07j, dtype=torch.complex128))
['Day = ', 1327]
(tensor(-7.1540e-07-9.7642e-07j, dtype=torch.complex128), tensor(-4.0247e-08-1.8755e-07j, dtype=torch.complex128))
['Day = ', 1328]
(tensor(-5.0203e-07-9.8777e-07j, dtype=torch.complex128), tensor(-2.4527e-08-1.9381e-07j, dtype=torch.complex128))
['Day = ', 1329]
(tensor(-4.4271e-07-1.1803e-06j, dtype=torch.complex128), tensor(-6.5697e-08-2.1307e-07j, dtype=torch.complex128))
['Day = ', 1330]
(tensor(-4.8908e-07-1.2346e-06j, dtype=torch.complex128), tensor(-1.1199e-07-2.1148e-07j, dtype=torch.complex128))
['Day = ', 1331]
(tensor(-3.7722e-07-1.1440e-06j, dtype=torch.complex128), tensor(-1.1334e-07

(tensor(-7.1540e-07-9.7642e-07j, dtype=torch.complex128), tensor(-4.0247e-08-1.8755e-07j, dtype=torch.complex128))
['Day = ', 1388]
(tensor(-5.0203e-07-9.8777e-07j, dtype=torch.complex128), tensor(-2.4527e-08-1.9381e-07j, dtype=torch.complex128))
['Day = ', 1389]
(tensor(-4.4271e-07-1.1803e-06j, dtype=torch.complex128), tensor(-6.5697e-08-2.1307e-07j, dtype=torch.complex128))
['Day = ', 1390]
(tensor(-4.8908e-07-1.2346e-06j, dtype=torch.complex128), tensor(-1.1199e-07-2.1148e-07j, dtype=torch.complex128))
['Day = ', 1391]
(tensor(-3.7722e-07-1.1440e-06j, dtype=torch.complex128), tensor(-1.1334e-07-2.3031e-07j, dtype=torch.complex128))
['Day = ', 1392]
(tensor(-2.4539e-07-1.2448e-06j, dtype=torch.complex128), tensor(-1.5523e-07-2.1954e-07j, dtype=torch.complex128))
['Day = ', 1393]
(tensor(-2.1770e-07-1.3528e-06j, dtype=torch.complex128), tensor(-1.5981e-07-2.3269e-07j, dtype=torch.complex128))
['Day = ', 1394]
(tensor(-2.7095e-07-1.4845e-06j, dtype=torch.complex128), tensor(-1.5080e-07

(tensor(-4.8908e-07-1.2346e-06j, dtype=torch.complex128), tensor(-1.1199e-07-2.1148e-07j, dtype=torch.complex128))
['Day = ', 1451]
(tensor(-3.7722e-07-1.1440e-06j, dtype=torch.complex128), tensor(-1.1334e-07-2.3031e-07j, dtype=torch.complex128))
['Day = ', 1452]
(tensor(-2.4539e-07-1.2448e-06j, dtype=torch.complex128), tensor(-1.5523e-07-2.1954e-07j, dtype=torch.complex128))
['Day = ', 1453]
(tensor(-2.1770e-07-1.3528e-06j, dtype=torch.complex128), tensor(-1.5981e-07-2.3269e-07j, dtype=torch.complex128))
['Day = ', 1454]
(tensor(-2.7095e-07-1.4845e-06j, dtype=torch.complex128), tensor(-1.5080e-07-2.2640e-07j, dtype=torch.complex128))
['Day = ', 1455]
(tensor(2.9567e-07-6.3008e-07j, dtype=torch.complex128), tensor(6.6558e-08-6.7375e-08j, dtype=torch.complex128))
['Day = ', 1456]
(tensor(-9.5554e-08-9.3223e-07j, dtype=torch.complex128), tensor(1.8942e-08-1.6881e-08j, dtype=torch.complex128))
['Day = ', 1457]
(tensor(-4.5622e-07-8.8174e-07j, dtype=torch.complex128), tensor(-3.1568e-08-8.

(tensor(-2.1770e-07-1.3528e-06j, dtype=torch.complex128), tensor(-1.5981e-07-2.3269e-07j, dtype=torch.complex128))
['Day = ', 1514]
(tensor(-2.7095e-07-1.4845e-06j, dtype=torch.complex128), tensor(-1.5080e-07-2.2640e-07j, dtype=torch.complex128))
['Day = ', 1515]
(tensor(2.9567e-07-6.3008e-07j, dtype=torch.complex128), tensor(6.6558e-08-6.7375e-08j, dtype=torch.complex128))
['Day = ', 1516]
(tensor(-9.5554e-08-9.3223e-07j, dtype=torch.complex128), tensor(1.8942e-08-1.6881e-08j, dtype=torch.complex128))
['Day = ', 1517]
(tensor(-4.5622e-07-8.8174e-07j, dtype=torch.complex128), tensor(-3.1568e-08-8.6623e-08j, dtype=torch.complex128))
['Day = ', 1518]
(tensor(-4.5959e-07-8.1709e-07j, dtype=torch.complex128), tensor(-3.8242e-08-1.5949e-07j, dtype=torch.complex128))
['Day = ', 1519]
(tensor(-4.2093e-07-9.6432e-07j, dtype=torch.complex128), tensor(-3.5105e-08-1.4263e-07j, dtype=torch.complex128))
['Day = ', 1520]
(tensor(-5.8640e-07-1.1361e-06j, dtype=torch.complex128), tensor(-2.3931e-08-1.

In [ ]:
## Write spectral data for possible restart
##
torch.save(zmn1,datapath+'zmn1.spectral.pt')
torch.save(zmn2,datapath+'zmn2.spectral.pt')
torch.save(dmn1,datapath+'dmn1.spectral.pt')
torch.save(dmn2,datapath+'dmn2.spectral.pt')
torch.save(tmn1,datapath+'tmn1.spectral.pt')
torch.save(tmn2,datapath+'tmn2.spectral.pt')
torch.save(qmn1,datapath+'qmn1.spectral.pt')
torch.save(qmn2,datapath+'qmn2.spectral.pt')